In [1]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold
import utils


In [2]:
from utils import Imputer, MainFeatureExtractor, SimpleStatisticsExtractor, GeoFeatureExtractor, UserFeatureExtractor, TargetEncodingExtractor

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/p.a.nikulin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
path = "data/"
data_start = pd.read_csv(path + 'train_events.csv')
video_start = pd.read_csv(path + 'video_info_v2.csv')
targets_start = pd.read_csv(path + 'train_targets.csv')

In [4]:
events, features = Imputer().fit_transform(data_start, video_start, targets_start)

for extractor in [MainFeatureExtractor(), SimpleStatisticsExtractor(), GeoFeatureExtractor(), UserFeatureExtractor()]:
    events, features = extractor.fit_transform(events, features)

features.shape

Applying SimpleStatisticsExtractor...


(180012, 174)

In [11]:
class CatboostEstimator:
    """
    Класс для обучения Catboost
    """

    def fit(self, X, y, ids, n_splits, cat_features, score):
        """
        Разбивает данные на k фолдов со стратификацией и обучает n_splits катбустов
        """
        self.one_model = False
        self.models = []
        scores = []
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        for ind, (train_index, val_index) in enumerate(skf.split(X, y)):
            X_train = X.loc[train_index]
            y_train = y.loc[train_index]
            X_val = X.loc[val_index]
            y_val = y.loc[val_index]

            if y.name == 'age_class':
                train_idx = ids.loc[train_index]
                train_idx.name = 'viewer_uid'

                y_train_idx = y_train.copy()
                y_train_idx.index = train_idx
                X_train['viewer_uid'] = train_idx
                val_idx = ids.loc[val_index]
                val_idx.name = 'viewer_uid'
                X_val['viewer_uid'] = val_idx
                
                target_enc_ext = utils.TargetEncodingExtractor()
                events_filtered_train = pd.merge(events, train_idx, on='viewer_uid', how='inner')
                target_enc_ext.fit(events_filtered_train, pd.merge(X_train, y_train_idx, on='viewer_uid', how='inner'))
                events_filtered_test = pd.merge(events, val_idx, on='viewer_uid', how='inner')

                X_train = target_enc_ext.transform(events_filtered_train, X_train).drop(columns=['viewer_uid'])
                X_val = target_enc_ext.transform(events_filtered_test, X_val).drop(columns=['viewer_uid'])

            model = CatBoostClassifier(cat_features=cat_features, verbose=500, iterations=1000)
            model.fit(X_train, y_train, verbose=500, eval_set=(X_val, y_val))
            
            self.models.append(model)
            y_pred = model.predict(X_val)
            scores.append(score(y_val, y_pred))
            print(f'model {ind}: score = {round(scores[-1], 4)}')
        
        scores = np.array(scores)
        print(f'mean score = {scores.mean().round(4)}, std = {scores.std().round(4)}')
        print(f'overall score = {(scores.mean() - scores.std()).round(4)}')
            
    
    def fit_select_features(self, X, y, cat_features, to_drop):
        """
        Обучает один катбуст и выполняет elect features
        """
        self.one_model = True
        
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        
        self.model = CatBoostClassifier(cat_features=cat_features, verbose=150, iterations=2000)
        
        self.model.select_features(X_train, y_train, verbose=500, eval_set=(X_val, y_val), steps=10,
                                  num_features_to_select=30, features_for_select=X.columns,
                                  algorithm='RecursiveByLossFunctionChange', train_final_model=True)

    def predict(self, X, cnt_classes):
        if self.one_model:
            return self.model.predict_proba(X)
        
        y_pred = np.zeros((X.shape[0], cnt_classes))

        for model in self.models:
            y_pred += model.predict_proba(X)
        y_pred /= cnt_classes
        y_pred = np.argmax(y_pred, axis=1)
        
        return y_pred

In [12]:
from sklearn.metrics import f1_score, accuracy_score

In [13]:
def score_sex(y_true, y_pred):
    return accuracy_score(y_true, y_pred)


def score_age(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')

In [14]:
cat_features = []
for i, col in enumerate(features.columns):
    if features[col].dtype in ['object', 'category']:
        cat_features.append(col)
        
cat_features

['region', 'favourite_cat']

In [15]:
catboost_sex = CatboostEstimator()
catboost_age = CatboostEstimator()

features_to_drop = [
    'viewer_uid',
    'age'
]

target_sex = 'sex'
target_age = 'age_class'

print('Sex model\n')

ids = features.reset_index()['viewer_uid']
ids.name = 'viewer_uid'
y_sex = features[target_sex]
y_sex.name = target_sex

y_age = features[target_age]
y_age.name = target_age

# catboost_sex.fit(features.drop(columns=features_to_drop + [target_sex] + [target_age]),
#                  y_sex,
#                  ids,
#                  n_splits=2,
#                  cat_features=cat_features,
#                  score=score_sex)

print('\n\n\nAge model\n')

catboost_age.fit(features.drop(columns=features_to_drop + [target_sex] + [target_age]),
                 y_age,
                 ids,
                 n_splits=2,
                 cat_features=cat_features,
                 score=score_age)

Sex model




Age model

Learning rate set to 0.118144
0:	learn: 1.3340863	test: 1.3349050	best: 1.3349050 (0)	total: 279ms	remaining: 4m 39s
500:	learn: 0.9964817	test: 1.0821731	best: 1.0821674 (478)	total: 57.1s	remaining: 56.9s
999:	learn: 0.9293448	test: 1.0845855	best: 1.0821437 (508)	total: 1m 54s	remaining: 0us

bestTest = 1.082143693
bestIteration = 508

Shrink model to first 509 iterations.
model 0: score = 0.4684
Learning rate set to 0.118144
0:	learn: 1.3358484	test: 1.3364706	best: 1.3364706 (0)	total: 142ms	remaining: 2m 21s
500:	learn: 0.9957295	test: 1.0840009	best: 1.0839145 (456)	total: 1m 2s	remaining: 1m 2s
999:	learn: 0.9286928	test: 1.0865387	best: 1.0839145 (456)	total: 2m	remaining: 0us

bestTest = 1.083914531
bestIteration = 456

Shrink model to first 457 iterations.
model 1: score = 0.4686
mean score = 0.4685, std = 0.0001
overall score = 0.4684


In [183]:
# Plot feature importances

def plot_importance(model, features, top=50):
    feature_importance = model.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5

    plt.figure(figsize=(12, 12))
    plt.barh(pos[-top:], feature_importance[sorted_idx][-top:], align='center')
    plt.yticks(pos[-top:], np.array(features.columns)[sorted_idx][-top:])
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')
    plt.show()
    
plot_importance(catboost_age.models[0], features.drop(columns=features_to_drop + [target_sex] + [target_age]))

In [10]:
import importlib
import utils
importlib.reload(utils)


<module 'utils' from '/Users/p.a.nikulin/ML/SocDemPrediction/utils/__init__.py'>